In [5]:
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral(Omentum)":"orange",
    "Brain-Caudate(basalganglia)":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed(Suprapubic)":"blue",
    "Skin-SunExposed(Lowerleg)":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m"
        }

SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral(Omentum)":"Adipose V.",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "Skin Unexposed",
    "Skin-SunExposed(Lowerleg)":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

import math
import numpy as np
import os
import pandas as pd
import random
import sys
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

tissu = sorted(COLORS.keys())
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t', header=None)
Motifs.columns = ['chrom','str.start','str.end','motif.len','motif']

In [44]:
A=[]
B=[]
All_eSTRs=[]
for Tissue in tissu: 
    
    CAUSE='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/CAVIAR_analysis.table'
    SCORE='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/strs_scorechr'
    LR = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/PQValues'
    ANOV = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/anova_wg.csv'
    #Anova
#    anova = pd.read_csv(ANOV, sep=',')
#    anova = anova[['chrom','gene','str.start','anova_pval','estr_fdr','esnp_fdr','delta_bic','delta_aic']]
    #LR
    lr = pd.read_csv(LR,sep='\t')
    #Causal 
    causal = pd.read_csv(CAUSE, sep='\t')
    causal['chrom']=causal['CHROM']
    #Scores
    Cs=[]
    for i in CHR:
        C=pd.read_csv(SCORE+str(i), sep='\t', header=None)
        Cs.append(C)
    scores=Cs[0]
    for i in range(1, len(Cs)):
        scores=scores.append(Cs[i]) 
    scores.columns=['chrom', 'gene', 'str.id', 'caviar.score']
    CA = scores; CA.index = CA['gene']        
    #Merge all in table
    Table = pd.merge(lr, CA, on=['chrom','gene','str.id'], how='left')
    print('LR:',lr.shape, 'causal scores:',CA.shape, 'Merged:',Table.shape)
    Table.index = Table['gene']
    Master = pd.merge(Table,causal, on=['chrom','gene'], how='left')
    print('With Caviar: ',Master.shape)

    #Gene names
    genes=Gene_table.loc[Gene_table['gene'].isin(list(Master['gene']))]
    genes=genes[['gene','chrom','gene.name']] 
    Master=pd.merge(Master, genes, on=['chrom','gene'])
    #motifs
    Master=pd.merge(Master, Motifs [['chrom','str.start','motif','str.end']], on=['chrom','str.start'])
    Master['top.str.rank']=Master['str.rank']
    #A.append(Master.loc[Master['gene.name']=='REL'] )
    #B.append(Master.loc[Master['gene.name']=='CSTB'] )
    Header = ['chrom','gene', 'str.id','p.wald', 'llqvalue', 'qvalue', 'caviar.score','top_str', 'top.str.score', 'beta', 'beta.se', 'num.strs.in.top5','llsignif', 'NTEST', 'significant', 'top_snp', 'top_snp_score','top.str.rank', 'motif','gene.name', 'str.start','str.end']
    Master = Master[Header].sort_values(['chrom','gene','str.start'])
#    Head=['chrom','gene','gene.name','str.id','str.start','best.str.start','significant','qvalue','best.str.score','top.variant','top.variant.score','anova_pval','delta_bic','delta_aic','p.wald','beta','NTEST','beta.se','estr_fdr','esnp_fdr','motif']
#    Master.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Master.table', sep='\t',  index=None)
    print('With gene name and motif : ',Master.shape, Tissue)
    break
#A = pd.concat(A) ##
#B = pd.concat(B) ##
#B['Tissues']=tissu
#B.index=B['Tissues']
#B['Tissues']=tissu
#B.index=B['Tissues']
#print('END!')
#NOT IN OUR DATA 
##   UGT1A1    VLDLR     MIF    PCA3  MXI1
#IN OUR DATA BUT NOT ESTRS
    #ARHG---       TRIB1    ARID1B   REL  
#IN OUR DATA with ESTRS
    #  CSTB
##   RUN The following
##   python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/STRs-vs-SNPs_effects/merge_causality_scores.py /storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/ WholeBlood,Cells-Transformedfibroblasts,Muscle-Skeletal,Lung,Artery-Tibial,Adipose-Subcutaneous,Esophagus-Mucosa causality > Merged_Best_causality.Table
##   To merge n select best by tissue

LR: (87891, 12) causal scores: (87780, 4) Merged: (87891, 13)
With Caviar:  (87891, 20)
With gene name and motif :  (87891, 22) Adipose-Subcutaneous


In [47]:
A = Master
B = A.loc[A['llsignif']==1]# [['p.wald','llqvalue','llsignif','qvalue','significant','caviar.score']]
lle = A.loc[A['llqvalue']<=0.1]
print(lle.shape, lle.loc[lle['qvalue']<=0.1].shape)

gle = A.loc[A['qvalue']<=0.1]
print(gle.shape, gle.loc[gle['llqvalue']<=0.1].shape)
gle.loc[gle['llqvalue']>=0.1][['chrom','str.start','gene.name','motif','p.wald','llqvalue','llsignif','qvalue','significant','caviar.score']].shape
Master.loc[Master['llqvalue']<=0.1].sort_values(['chrom','gene','str.start'])

(4516, 22) (1752, 22)
(1789, 22) (1752, 22)


,chrom,gene,str.id,p.wald,llqvalue,qvalue,caviar.score,top_str,top.str.score,beta,...,llsignif,NTEST,significant,top_snp,top_snp_score,top.str.rank,motif,gene.name,str.start,str.end
39,chr1,ENSG00000000971.11,STR_196527511,3.714764e-03,0.072028,0.134687,1.165190e-02,STR_196527511,1.165190e-02,0.185087,...,1,5.0,0.0,SNP_196569681,0.099115,22.0,AC,CFH,196527511,196527533
53,chr1,ENSG00000001460.13,STR_24722732,1.676038e-11,0.001010,0.002077,7.261270e-05,STR_24722732,7.261270e-05,-0.400286,...,1,2.0,1.0,SNP_24781247,0.411762,36.0,AGAT,STPG1,24722732,24722801
54,chr1,ENSG00000001461.12,STR_24722732,5.131596e-05,0.002623,0.002763,6.352310e-03,STR_24722732,6.352310e-03,-0.247463,...,1,2.0,1.0,SNP_24712349,0.042790,34.0,AGAT,NIPAL3,24722732,24722801
235,chr1,ENSG00000010803.12,STR_41744245,1.037732e-03,0.028726,0.026009,1.480850e-03,STR_41744245,1.480850e-03,0.203064,...,1,2.0,1.0,SNP_41468135,0.032573,101.0,AC,SCMH1,41744245,41744289
553,chr1,ENSG00000037637.6,STR_16709651,3.809107e-03,0.073244,0.116993,6.253020e-03,STR_16709651,6.253020e-03,-0.191750,...,1,4.0,0.0,SNP_16535804,0.026828,101.0,AAAG,FBXO42,16709651,16709731
584,chr1,ENSG00000043514.11,STR_40294139,1.199059e-03,0.031948,0.049832,4.529160e-06,STR_40294139,4.529160e-06,0.211391,...,1,4.0,1.0,SNP_40303627,0.112049,59.0,AC,TRIT1,40294139,40294181
558,chr1,ENSG00000053371.8,STR_19642848,4.593745e-03,0.083357,0.110011,1.657330e-02,STR_19642848,1.657330e-02,-0.180872,...,1,3.0,0.0,SNP_19672999,0.032481,6.0,AC,AKR7A2,19642848,19642888
668,chr1,ENSG00000054267.16,STR_235298544,3.514599e-03,0.069332,0.146547,3.812590e-02,STR_235298544,3.812590e-02,-0.188095,...,1,6.0,0.0,SNP_235465034,0.015203,2.0,A,ARID4B,235298544,235298554
710,chr1,ENSG00000054282.11,STR_243394402,5.337398e-04,0.017200,0.015152,2.891300e-04,STR_243394402,2.891300e-04,-0.224228,...,1,2.0,1.0,SNP_243418182,0.089880,73.0,A,SDCCAG8,243394402,243394412
955,chr1,ENSG00000066056.9,STR_43761506,7.041456e-06,0.001057,0.002085,8.837170e-04,STR_43761506,8.837170e-04,-0.300328,...,1,4.0,1.0,SNP_43766426,0.135401,86.0,AC,TIE1,43761506,43761555


In [5]:
#A = pd.concat(A) ##UGT1A1 
#B = pd.concat(B) ##MIF
B['Tissues']=tissu
B.index=B['Tissues']
B[['chrom','gene','gene.name','best.str.start', 'significant','qvalue', 'best.str.score','top.variant','top.variant.score','anova_pval','delta_bic','delta_aic','motif']]

,chrom,gene,gene.name,best.str.start,significant,qvalue,best.str.score,top.variant,top.variant.score,anova_pval,delta_bic,delta_aic,motif
Tissues,,,,,,,,,,,,,
Adipose-Subcutaneous,chr21,ENSG00000160213.5,CSTB,45196326,1,0.026765,0.001256,SNP_45233500,0.098794,3.623376e-01,-4.727337,-1.158992,CCCCG
Adipose-Visceral(Omentum),chr21,ENSG00000160213.5,CSTB,45196326,1,0.009244,0.164337,STR_45196326,0.164337,9.073061e-03,1.756690,4.944076,CCCCG
Artery-Aorta,chr21,ENSG00000160213.5,CSTB,45196326,1,0.013575,0.143539,STR_45196326,0.143539,8.581903e-04,6.091580,9.322689,CCCCG
Artery-Tibial,chr21,ENSG00000160213.5,CSTB,45196326,1,0.002820,0.999982,STR_45196326,0.999982,2.076818e-07,21.715432,25.313854,CCCCG
Brain-Caudate(basalganglia),chr21,ENSG00000160213.5,CSTB,45170834,0,0.724641,0.006582,SNP_45176268,0.075583,2.722182e-01,-3.396894,-0.752503,AC
Brain-Cerebellum,chr21,ENSG00000160213.5,CSTB,45277167,0,1.000000,0.004736,SNP_45243026,0.079672,3.317642e-01,-3.629022,-1.023852,AAC
Cells-Transformedfibroblasts,chr21,ENSG00000160213.5,CSTB,45196326,1,0.003467,0.998661,STR_45196326,0.998661,1.427390e-06,18.250389,21.625667,CCCCG
Esophagus-Mucosa,chr21,ENSG00000160213.5,CSTB,45196326,0,0.911693,0.005293,SNP_45158689,0.026499,1.423925e-01,-3.326554,0.182834,CCCCG
Esophagus-Muscularis,chr21,ENSG00000160213.5,CSTB,45196326,1,0.004218,0.990433,STR_45196326,0.990433,4.112573e-05,11.787493,15.110503,CCCCG


In [25]:
"','".join(Head)

"chrom','gene','gene.name','str.id','str.start','best.str.start','significant','qvalue','best.str.score','top.variant','top.variant.score','anova_pval','delta_bic','delta_aic','p.wald','beta','NTEST','beta.se','estr_fdr','esnp_fdr','motif"

In [ ]:
Header = ['chrom','gene', 'gene.name', 'str.start', 'str.id', 'beta', 'beta.se','p.wald', 'llqvalue','llsignif', 'NTEST', 'qvalue', 'significant','caviar.score', 'num.strs.in.top5', 'top_snp', 'top_snp_score','top_str', 'top.str.score', 'str.rank', 'motif','str.end']



In [ ]:
#Heat scatter plots showing the heritability of each eQTL 
#apportioned to the most significant STR in the cis window 
#x = axis h2_str y = the additive effect from the top 100 most significant SNVs 

Master['h2b']= Master["cis_snp_h2"] + Master["cis_str_h2"]
MM1 = Master.loc[Master['h2b']<=.5]
ESTR = Master.loc[Master['qvalue']<0.1]
#plt.scatter(ESTR['cis_str_h2'],ESTR['h2b'])
sns.jointplot(x="cis_str_h2", y="h2b", data=MM1, kind="kde", color="g")
plt.show()
print (Master.shape, ' After restricting to h2b<=1 ',MM1.shape)